# Pretext
In a world ever more interconnected, the marriage of machine learning and agriculture gains profound significance. With our global population swelling, the study of crop yield becomes paramount. The intricate dance of weather patterns, chemical treatments, and historical insights determines agricultural success. Unlocking this puzzle holds the key to food security and resilience against climate flux.     

This project squarely addresses this challenge by deploying machine learning to predict the top 10 most-consumed crops worldwide. These essential staples like corn, wheat, and rice, form the bedrock of human sustenance. By harnessing the power of regression techniques, we develop a path to foresee yields, empowering farmers, as they can optimise resources.

In this notebook, I have aimed to provide deep, insightful analysis, and visualisation, unveiling meaningful information that will be of significant value, and have also finally built a regression model, that can predict yields on unseen test data, with a whopping R2 score of about 0.972!

<div style="background-color: #fce5cd; padding: 20px; border-radius: 10px;">
    <p style="font-size: 18px; text-align: center;"><em>Your support fuels inspiration!</em></p>
    <p style="font-size: 16px; text-align: center;">🌟 If this Notebook has intrigued you, I humbly invite you to join in celebrating the journey. An upvote is a resounding cheer, a way to say "Bravo!"</p>
    <p style="font-size: 16px; text-align: center;">Let's build a connection – a bridge for ideas to meet up. Your engagement encourages, and I extend a warm welcome to connect.</p>
    <p style="font-size: 16px; text-align: center;">Together, let's revel in the journey of exploration. 🚀🤝</p>
</div>



# Loading and reading the datasets

After importing required libraries, crops yield of ten most consumed crops around the world was downloaded from FAO webiste.The collected data include country, item, year starting from 1961 to 2016 and yield value

In [2]:
#importing the basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#ingesting the datasets 
pest_df=pd.read_csv('Datasets/Crop_Yield_dataset/pesticides.csv')
rain_df=pd.read_csv('Datasets/Crop_Yield_dataset/rainfall.csv')
temp_df=pd.read_csv('Datasets/Crop_Yield_dataset/temp.csv/temp.csv')
yield_df=pd.read_csv('Datasets/Crop_Yield_dataset/yield.csv/yield.csv')
data_frames=[pest_df,rain_df,temp_df,yield_df]

#reading a sample set of rows for all the ingested datasets 
for df in data_frames:
    print('A sample set of rows for {} is:\n'.format(df))
    print(df.sample(6))

A sample set of rows for               Domain      Area Element                Item  Year  \
0     Pesticides Use   Albania     Use  Pesticides (total)  1990   
1     Pesticides Use   Albania     Use  Pesticides (total)  1991   
2     Pesticides Use   Albania     Use  Pesticides (total)  1992   
3     Pesticides Use   Albania     Use  Pesticides (total)  1993   
4     Pesticides Use   Albania     Use  Pesticides (total)  1994   
...              ...       ...     ...                 ...   ...   
4344  Pesticides Use  Zimbabwe     Use  Pesticides (total)  2012   
4345  Pesticides Use  Zimbabwe     Use  Pesticides (total)  2013   
4346  Pesticides Use  Zimbabwe     Use  Pesticides (total)  2014   
4347  Pesticides Use  Zimbabwe     Use  Pesticides (total)  2015   
4348  Pesticides Use  Zimbabwe     Use  Pesticides (total)  2016   

                              Unit    Value  
0     tonnes of active ingredients   121.00  
1     tonnes of active ingredients   121.00  
2     tonnes of acti

# Performing Feature Engineering and Null value imputation

In [3]:
# Dropping unnecessary columns, as they wont be of anu help to find patterns in our
# data
pest_df=pest_df.drop(['Unit','Domain','Element','Item'],axis=1)
yield_df=yield_df.drop(['Domain Code','Domain','Area Code','Element Code','Item Code','Year Code','Unit'],axis=1)
pest_df.head()

,Area,Year,Value
0,Albania,1990,121.0
1,Albania,1991,121.0
2,Albania,1992,121.0
3,Albania,1993,121.0
4,Albania,1994,201.0


In [4]:
yield_df.head()

,Area,Element,Item,Year,Value
0,Afghanistan,Yield,Maize,1961,14000
1,Afghanistan,Yield,Maize,1962,14000
2,Afghanistan,Yield,Maize,1963,14260
3,Afghanistan,Yield,Maize,1964,14257
4,Afghanistan,Yield,Maize,1965,14400


In [5]:
yield_df.columns

Index(['Area', 'Element', 'Item', 'Year', 'Value'], dtype='object')

In [6]:
rain_df.rename(columns = {' Area':'Area'},inplace = True)
for df in data_frames:
    print(df.columns)

Index(['Domain', 'Area', 'Element', 'Item', 'Year', 'Unit', 'Value'], dtype='object')
Index(['Area', 'Year', 'average_rain_fall_mm_per_year'], dtype='object')
Index(['year', 'country', 'avg_temp'], dtype='object')
Index(['Domain Code', 'Domain', 'Area Code', 'Area', 'Element Code', 'Element',
       'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value'],
      dtype='object')


In [7]:
temp_df.rename(columns = {'year':'Year','country':'Area'},inplace = True)
temp_df.columns

Index(['Year', 'Area', 'avg_temp'], dtype='object')

In [8]:
# Merging our datasets into 1 single dataframe
pr=pd.merge(pest_df,rain_df,on=['Year','Area'])
prt=pd.merge(pr,temp_df,on=['Year','Area'])
prty=pd.merge(yield_df,prt,on=['Year','Area'])
print(prty.columns)
prty.sample(10)

Index(['Area', 'Element', 'Item', 'Year', 'Value_x', 'Value_y',
       'average_rain_fall_mm_per_year', 'avg_temp'],
      dtype='object')


,Area,Element,Item,Year,Value_x,Value_y,average_rain_fall_mm_per_year,avg_temp
2713,Brazil,Yield,Sweet potatoes,1991,101504,58349.44,1761,21.43
13278,India,Yield,Soybeans,2006,10628,37423.00,1083,27.15
21589,Niger,Yield,Sorghum,2002,2989,29.44,151,29.71
15211,Indonesia,Yield,Maize,2009,42372,1597.00,2702,27.31
12074,India,Yield,Wheat,1998,24852,49157.00,1083,26.01
563,Argentina,Yield,Soybeans,2001,25846,63700.17,591,17.71
5635,Canada,Yield,Maize,1992,56927,29387.00,537,3.69
12523,India,Yield,"Rice, paddy",2001,31158,43720.04,1083,26.32
3042,Brazil,Yield,Yams,1994,91667,84312.78,1761,25.42
11315,India,Yield,Sorghum,1994,7786,61357.00,1083,25.71


In [9]:
prty.rename(columns={'Value_y':'pesticides_tonnes','Value_x':'hg/ha_yield'},inplace=True)
print(prty.columns)

Index(['Area', 'Element', 'Item', 'Year', 'hg/ha_yield', 'pesticides_tonnes',
       'average_rain_fall_mm_per_year', 'avg_temp'],
      dtype='object')


### Descriptive Data Analysis

In [10]:
prty.shape

(28248, 8)

In [11]:
prty.describe()

,Year,hg/ha_yield,pesticides_tonnes,avg_temp
count,28248.000000,28248.000000,28248.000000,28248.000000
mean,2001.541950,77047.863282,37069.136973,20.543722
std,7.052997,84950.194454,59954.787836,6.311828
min,1990.000000,50.000000,0.040000,1.300000
25%,1995.000000,19918.750000,1695.710000,16.710000
50%,2001.000000,38295.000000,17517.760000,21.510000
75%,2008.000000,104598.250000,48687.880000,26.000000
max,2013.000000,501412.000000,367778.000000,30.650000


In [12]:
prty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28248 entries, 0 to 28247
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Area                           28248 non-null  object 
 1   Element                        28248 non-null  object 
 2   Item                           28248 non-null  object 
 3   Year                           28248 non-null  int64  
 4   hg/ha_yield                    28248 non-null  int64  
 5   pesticides_tonnes              28248 non-null  float64
 6   average_rain_fall_mm_per_year  28248 non-null  object 
 7   avg_temp                       28248 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.7+ MB


**Here, the data for our rainfall is of Object data type but is desired in integer format**

In [13]:
prty.isnull().sum().sum()/prty.shape[0]

0.0

**There is no need of null value imputation as no null cells**

In [14]:
#converting the datatype of rainfall data to a desirable type
prty['average_rain_fall_mm_per_year'] = prty['average_rain_fall_mm_per_year'].replace('..',np.nan)
prty['average_rain_fall_mm_per_year'] = prty['average_rain_fall_mm_per_year'].astype('float')
prty=prty.dropna()
#filtering out the numerical and categorical columns, that will be useful for our 
#feature scaling and EDA
num_cols = [i for i in prty.columns if (prty[i].dtype == 'float64' or prty[i].dtype == 'int64')]
cat_cols = [i for i in prty.columns if (i not in num_cols) and i != 'hg/ha_yield']
print(num_cols)
print(cat_cols)

['Year', 'hg/ha_yield', 'pesticides_tonnes', 'average_rain_fall_mm_per_year', 'avg_temp']
['Area', 'Element', 'Item']


In [15]:
prty['Item']=prty['Item'].replace('Rice,paddy','Rice')
prty['Item']=prty['Item'].replace('Plantains and others','Plantains and Bananas')
prty['Item']=prty['Item'].replace('Potatoes','Apple')
prty['Item']=prty['Item'].replace('Wheat','lentil')
prty['Item']=prty['Item'].replace('Yams','mungbean')

In [16]:
prty.to_csv('Datasets/Crop_Yield_dataset/prty.csv')

# Exploratory Data Analysis
Here onwards, we have some detailed visual analysis of several patterns in our data. We have used multiple data analytics and visualization techniques and plotted several insightful results! Stay glued for the upcoming, intriguing insights !

In [ ]:
plt.figure(figsize=(12, 16))
for i, col in enumerate(num_cols, 1):
    plt.subplot(3, 2, i)
    sns.boxplot(data=prty[col])
    plt.title(f'Box Plot of {col}')
    plt.ylabel('Value')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='Item',data=prty)
plt.title('Countplot of Item vs its count')
plt.xlabel('Crops',fontsize=18,loc='center')
plt.ylabel('Count',fontsize=18,loc='center')
plt.xticks(rotation=45)
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(18, 22))
sns.lineplot(x = "pesticides_tonnes", y = "hg/ha_yield", hue = "Item", data = prty, ax=axes[0], legend = True)
axes[0].tick_params(axis='x', rotation=45)
axes[0].set_ylabel('Average Yield')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.lineplot(x = "average_rain_fall_mm_per_year", y = "hg/ha_yield", hue = "Item", data = prty, ax=axes[1], legend = True)
axes[1].tick_params(axis='x', rotation=45)
axes[1].set_ylabel('Average Yield')
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.lineplot(x = "avg_temp", y = "hg/ha_yield", hue = "Item", data = prty, ax=axes[2], legend = True)
axes[2].tick_params(axis='x', rotation=45)
axes[2].set_ylabel('Average Yield')
axes[2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
df=prty.copy()
df['yield_rainfall_ratio'] = df['hg/ha_yield'] / df['average_rain_fall_mm_per_year']

top_10_crops = df.groupby('Item')['yield_rainfall_ratio'].mean().sort_values(ascending=False).head(10).index

# Filter the data to only include the top 10 crops
top_10_data = df[df['Item'].isin(top_10_crops)]

sns.barplot(data=top_10_data, x='Item', y='yield_rainfall_ratio', order=top_10_crops)
plt.xlabel('Crops',fontsize=18)
plt.ylabel('Yield/Average Rainfall',fontsize=18)
plt.xticks(rotation=45)
plt.show()

In [ ]:
df=prty.copy()
df['yield_rainfall_ratio'] = df['hg/ha_yield'] / df['average_rain_fall_mm_per_year']

top_10_countries = df.groupby('Area')['yield_rainfall_ratio'].mean().sort_values(ascending=False).head(10).index
top_10_data = df[df['Area'].isin(top_10_countries)]

sns.boxplot(data=top_10_data, x='Area', y='yield_rainfall_ratio', order=top_10_countries)
plt.xlabel('Countries',fontsize=18)
plt.ylabel('Yield/Average Rainfall',fontsize=18)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
sns.heatmap(prty[num_cols].corr(), annot=True,linewidth=.5,cmap='crest')
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(18, 22))

sns.scatterplot(x = "pesticides_tonnes", y = "hg/ha_yield", hue = "Item", data = prty, ax=axes[0], legend = True)
axes[0].tick_params(axis='x', rotation=45)
axes[0].set_ylabel('Average Yield')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.scatterplot(x = "average_rain_fall_mm_per_year", y = "hg/ha_yield", hue = "Item", data = prty, ax=axes[1], legend = True)
axes[1].tick_params(axis='x', rotation=45)
axes[1].set_ylabel('Average Yield')
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.scatterplot(x = "avg_temp", y = "hg/ha_yield", hue = "Item", data = prty, ax=axes[2], legend = True)
axes[2].tick_params(axis='x', rotation=45)
axes[2].set_ylabel('Average Yield')
axes[2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
prty.columns

# Data Preprocessing

In [29]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder

# First, we have performed one hot encoding on the categorical features, and then
# performed standard scaling on all the features
prty=prty.drop('Year',axis=True)
X,y=prty.drop('hg/ha_yield',axis=1),prty['hg/ha_yield']
# X.shape, y.shape
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Assuming 'prty' is your DataFrame
X, y = prty.drop('hg/ha_yield', axis=1), prty['hg/ha_yield']

# Define the categorical columns you want to label encode
cat_cols = ['Item', 'Area', 'Element']

# Initialize LabelEncoder for each categorical column
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])  # Transform the column in-place
    label_encoders[col] = le  # Save the label encoder for later
    
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [69]:
X[1992]

array([-1.43567765,  0.        , -1.12911544, -0.61030202,  0.20138645,
        0.82183515])

In [30]:
X.shape, y.shape

((28242, 6), (28242,))

In [53]:
joblib.dump(label_encoders, 'models/label_encoders.pkl')
joblib.dump(scaler, 'models/scaler.pkl')

['models/scaler.pkl']

In [32]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.20,shuffle=True)
print(X_train.shape,X_test.shape)

(22593, 6) (5649, 6)


# Building our Initial model

In [24]:
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

poly = PolynomialFeatures(degree=2, include_bias=False)  # You can adjust the degree as needed
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Fit a Linear Regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train_poly, y_train)

# Predict on the test set
y_pred = lin_reg.predict(X_test_poly)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(mse,' ',r2)

3686681679.489396   0.49174960016152847


# Trying multiple models and Hyperparameter Tuning 

In [26]:
model_names=['svm_regressor','random_forest_regressor','lasso_regressor','XGBoost_Regressor']

svr=SVR(kernel='rbf', gamma='auto')
random_forest=RandomForestRegressor()
lasso_regressor = Lasso(alpha=1.0, random_state=42,max_iter=3000)
xgb_regressor = xgb.XGBRegressor(n_estimators=100, random_state=42)

models = [svr, random_forest, lasso_regressor, xgb_regressor]

model_params = [
    {},  # SVR doesn't require hyperparameters here
    {'n_estimators': [10, 50, 100]},  # RandomForestRegressor parameters
    {'alpha': [0.1, 1.0, 10.0]},  # Lasso parameters
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.5, 1.0]}  # XGBRegressor parameters
]

In [ ]:
# Here, we have performed hyperparameter tuning on multiple regression models
# to finally find out the best model

scores = []
best_estimators = {}

for name, model, params in zip(model_names, models, model_params):
#     pipe = make_pipeline(StandardScaler(), model)
    clf = GridSearchCV(model, params, cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[model] = clf.best_estimator_

res = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
res

In [33]:
best_model=RandomForestRegressor(n_estimators=100)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

R-squared: 0.9726740054808992


In [34]:
import joblib

model = best_model
# Save the model to a file
joblib.dump(model, 'models/yield_predictor.pkl')

['models/yield_predictor.pkl']

In [52]:
#model = joblib.load('models/yield_predictor.pkl')
#y_pred1 = pd.DataFrame()
#best_model.predict(X_test.loc[0])
y_test - best_model.predict(X_test)

25570    -1861.10
18119    -3748.03
25613    -1199.43
6821     10140.22
18150      831.62
           ...   
25033     5950.56
5514     12586.97
11399        0.00
16694     -119.09
3883       204.19
Name: hg/ha_yield, Length: 5649, dtype: float64

In [ ]:
#plotting the results of our model, against the original results
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.7, color='blue', label='Predicted')
sns.scatterplot(x=y_test, y=y_test, alpha=0.7, color='red', label='Actual')
plt.xlabel("Actual Values (y_test)")
plt.ylabel("Predicted Values (y_pred)")
plt.title("Actual vs. Predicted Values")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system names
        if name == "PIL":
            name = "Pillow"
        elif name == "sklearn":
            name = "scikit-learn"

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))